In [1]:
import time
import requests
import urllib
import json
from datetime import datetime
import dateutil.parser
from splinter import Browser
from tosConfig import consumerKey, passWord, userName
from tdAuthen import accesstoken

In [18]:
authen_data = accesstoken(consumerKey = consumerKey, passWord = passWord, userName = userName)

Pulled Code, grabbing access token....


In [19]:
import datetime
def unix_time_millis(dt):
    epoch = datetime.datetime.utcfromtimestamp(0)
    return (dt - epoch).total_seconds() * 1000.0

In [20]:
# define endpoint which is the User Principles emdpoint

endpoint = 'https://api.tdameritrade.com/v1/userprincipals'
headers = {'Authorization':'Bearer {}'.format(authen_data['access_token'])}

# this endpoint, requires fields which are separated by ','
params = {'fields':'streamerSubscriptionKeys,streamerConnectionInfo'}

# make a request
content = requests.get(url = endpoint, params = params, headers = headers)
userPrincipalsResponse = content.json()

# we need to get the timestamp in order to make our next request, but it needs to be parsed
tokenTimeStamp = userPrincipalsResponse['streamerInfo']['tokenTimestamp']
date = dateutil.parser.parse(tokenTimeStamp, ignoretz = True)
tokenTimeStampAsMs = unix_time_millis(date)

# we need to define our credentials that we will need to make our stream
credentials = {"userid": userPrincipalsResponse['accounts'][0]['accountId'],
               "token": userPrincipalsResponse['streamerInfo']['token'],
               "company": userPrincipalsResponse['accounts'][0]['company'],
               "segment": userPrincipalsResponse['accounts'][0]['segment'],
               "cddomain": userPrincipalsResponse['accounts'][0]['accountCdDomainId'],
               "usergroup": userPrincipalsResponse['streamerInfo']['userGroup'],
               "accesslevel":userPrincipalsResponse['streamerInfo']['accessLevel'],
               "authorized": "Y",
               "timestamp": int(tokenTimeStampAsMs),
               "appid": userPrincipalsResponse['streamerInfo']['appId'],
               "acl": userPrincipalsResponse['streamerInfo']['acl'] }

In [21]:
# define a request
login_request = {"requests": [{"service": "ADMIN",
                              "requestid": "0",  
                              "command": "LOGIN",
                              "account": userPrincipalsResponse['accounts'][0]['accountId'],
                              "source": userPrincipalsResponse['streamerInfo']['appId'],
                              "parameters": {"credential": urllib.parse.urlencode(credentials),
                                             "token": userPrincipalsResponse['streamerInfo']['token'],
                                             "version": "1.0"}}]}


# define a request for different data sources
data_request= {"requests": [{"service": "ACTIVES_NASDAQ", 
                             "requestid": "1", 
                             "command": "SUBS", 
                             "account": userPrincipalsResponse['accounts'][0]['accountId'], 
                             "source": userPrincipalsResponse['streamerInfo']['appId'], 
                             "parameters": {"keys": "NASDAQ-60", 
                                            "fields": "0,1"}},
                            {"service": "LEVELONE_FUTURES",
                             "requestid": "2",
                             "command": "SUBS",
                             "account": userPrincipalsResponse['accounts'][0]['accountId'],
                             "source": userPrincipalsResponse['streamerInfo']['appId'],
                             "parameters": {"keys": "/ES",
                                            "fields": "0,1,2,3,4"}}]}


# create it into a JSON string, as the API expects a JSON string.
login_encoded = json.dumps(login_request)
data_encoded = json.dumps(data_request)

In [22]:
import websockets
import asyncio
import pyodbc

class WebSocketClient():

    def __init__(self):
        self.data_holder = []
        self.file = open('td_ameritrade_data.txt', 'a')
        self.cnxn = None
        self.crsr = None
        
    def database_connect(self):
        
        # define the server and the database, YOU WILL NEED TO CHANGE THIS TO YOUR OWN DATABASE AND SERVER
        server = 'YOUR_SERVER' 
        database = 'YOUR_DATABASE'  
        sql_driver = '{ODBC Driver 17 for SQL Server}'

        # define our connection, autocommit MUST BE SET TO TRUE, also we can edit data.
        self.cnxn = pyodbc.connect(driver = sql_driver, 
                                   server = server, 
                                   database = database, 
                                   trusted_connection ='yes')

        self.crsr = self.cnxn.cursor()
        
    def database_insert(self, query, data_tuple):   
        
        # execute the query, commit the changes, and close the connection
        self.crsr.execute(query, data_tuple)
        self.cnxn.commit()
        self.cnxn.close()
        
        print('Data has been successfully inserted into the database.')

    async def connect(self):
        '''
            Connecting to webSocket server
            websockets.client.connect returns a WebSocketClientProtocol, which is used to send and receive messages
        '''
        
        # define the URI of the data stream, and connect to it.
        uri = "wss://" + userPrincipalsResponse['streamerInfo']['streamerSocketUrl'] + "/ws"
        self.connection = await websockets.client.connect(uri)
        
        # if all goes well, let the user know.
        if self.connection.open:
            print('Connection established. Client correctly connected')
            return self.connection


    async def sendMessage(self, message):
        '''
            Sending message to webSocket server
        '''
        await self.connection.send(message)
        

    async def receiveMessage(self, connection):
        '''
            Receiving all server messages and handle them
        '''
        while True:
            try:
                
                # grab and decode the message
                message = await connection.recv()                
                message_decoded = json.loads(message)
                
                # prepare data for insertion, connect to database
                query = "INSERT INTO td_service_data (service, timestamp, command) VALUES (?,?,?);"
                self.database_connect()
                
                # check if the response contains a key called data if so then it contains the info we want to insert.
                if 'data' in message_decoded.keys():
                    
                    # grab the data
                    data = message_decoded['data'][0]
                    data_tuple = (data['service'], str(data['timestamp']), data['command'])
                    
                    # insert the data
                    self.database_insert(query, data_tuple)
                    
                print('-'*20)
                print('Received message from server: ' + str(message))
                
            except websockets.exceptions.ConnectionClosed:            
                print('Connection with server closed')
                break

                
    async def heartbeat(self, connection):
        '''
            Sending heartbeat to server every 5 seconds
            Ping - pong messages to verify connection is alive
        '''
        while True:
            try:
                await connection.send('ping')
                await asyncio.sleep(5)
            except websockets.exceptions.ConnectionClosed:
                print('Connection with server closed')
                break

In [23]:
import nest_asyncio
from websockets import client
nest_asyncio.apply()


if __name__ == '__main__':
    
    # Creating client object
    client = WebSocketClient()
    
    loop = asyncio.get_event_loop()
    
    # Start connection and get client connection protocol
    connection = loop.run_until_complete(client.connect())
    
    # Start listener and heartbeat 
    tasks = [asyncio.ensure_future(client.receiveMessage(connection)),
             asyncio.ensure_future(client.sendMessage(login_encoded)),
             asyncio.ensure_future(client.receiveMessage(connection)),
             asyncio.ensure_future(client.sendMessage(data_encoded)),
             asyncio.ensure_future(client.receiveMessage(connection))]

    loop.run_until_complete(asyncio.wait(tasks))

NegotiationError: Unsupported extension: name = permessage-deflate, params = []